<a href="https://colab.research.google.com/github/priyanshu3042/rag-loan-chatbot/blob/main/Untitled40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q faiss-cpu sentence-transformers transformers

from google.colab import files
uploaded = files.upload()

import pandas as pd
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

docs = df.astype(str).apply(lambda row: ' | '.join(row), axis=1).tolist()
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embed_model.encode(docs, convert_to_numpy=True)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)
from transformers import pipeline

# Load public LLM
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# RAG chatbot function
def rag_chatbot(question, top_k=5):
    q_embed = embed_model.encode([question])
    _, indices = index.search(q_embed, top_k)
    context = "\n".join([docs[i] for i in indices[0]])
    prompt = f"Answer the question based on the context:\n\n{context}\n\nQuestion: {question}"
    result = qa_pipeline(prompt, max_new_tokens=150)
    return result[0]["generated_text"].strip()

# 🔁 Ask multiple questions
while True:
    question = input("\n💬 Ask a question (or type 'exit' to quit): ")
    if question.lower() == "exit":
        print("Chatbot ended.")
        break
    answer = rag_chatbot(question)
    print("🤖 Answer:", answer)



Saving Training Dataset.csv to Training Dataset (1).csv


Device set to use cpu



💬 Ask a question (or type 'exit' to quit): "What is the average loan amount in the dataset?"
🤖 Answer: 69.0

💬 Ask a question (or type 'exit' to quit):  "How does applicant income relate to loan approval?"
🤖 Answer: 3+ | Not Graduate | No | 3522 | 0.0 | 81.0 | 180.0 | 1.0 | Rural | N LP002517 | Male | Yes | 1 | Not Graduate | No | 2653 | 1500.0 | 113.0 | 180.0 | 0.0 | Rural | N LP001207 | Male | Yes | 0 | Not Graduate | Yes | 2609 | 3449.0 | 165.0 | 180.0 | 0.0 | Rural | N LP002941 | Male | Yes | 2 | Not Graduate | Yes | 6383 | 1000.0 | 187.0 | 360.0 | 1.0 | Rural | N

💬 Ask a question (or type 'exit' to quit):  "Is credit history important for getting a loan approved?"
🤖 Answer: Credit history is important for getting a loan approved.

💬 Ask a question (or type 'exit' to quit):  "What types of property area are most common?"
🤖 Answer: Rural

💬 Ask a question (or type 'exit' to quit): "Are married applicants more likely to be approved?"
🤖 Answer: LP001586 | Male | Yes | 0 | Not Gradua